# Mounting & Importing

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
import os
import zipfile
import pandas as pd
import numpy as np

# Unzipping

In [4]:
dir = '/content/drive/MyDrive/Quinto_Anio/TESIS_Eugenia_Berrino/Part_II_DS/DS_VAL_OD/unzipped_v2'
zip_ref = zipfile.ZipFile('/content/drive/MyDrive/Quinto_Anio/TESIS_Eugenia_Berrino/Part_II_DS/DS_VAL_OD/Euge_v2.zip', 'r')
zip_ref.extractall(path=dir)
zip_ref.close()

# Annotations

In [7]:
df = pd.read_csv('/content/drive/MyDrive/Quinto_Anio/TESIS_Eugenia_Berrino/Part_II_DS/DS_VAL_OD/unzipped/Euge/test_hiba.csv')
df

,file_name,class_name,height,width,label_level
0,14e9ae6ea30d11ebacbcf48e3885516d,NoduloMasa-Consolidacion,2048,2500,mask
1,14e9ae6fa30d11eb904cf48e3885516d,Atelectasia,2735,2802,mask
2,14e9ae70a30d11eb9ccaf48e3885516d,Consolidacion,3054,3056,mask
3,14e9ae71a30d11ebbffff48e3885516d,NoduloMasa,2735,2802,mask
4,14e9ae72a30d11eba183f48e3885516d,PatronIntersticial-PatronIntersticial-Lesiones...,2542,2740,mask
...,...,...,...,...,...
1317,14ed7f28a30d11eb905bf48e3885516d,NaN,2495,2736,nofinding
1318,14ed7f29a30d11ebb900f48e3885516d,NaN,2735,2802,nofinding
1319,14ed7f2aa30d11ebb8a8f48e3885516d,NaN,2801,2802,nofinding
1320,14ed7f2ba30d11eba5b1f48e3885516d,NaN,2619,2458,nofinding


In [ ]:
df2 = df[~df.class_name.isnull()]

In [ ]:
df[df.class_name.isnull()]

,file_name,class_name,height,width,label_level
233,14e9fce8a30d11eb9e14f48e3885516d,NaN,2677,2342,nofinding
234,14e9fce9a30d11eb951df48e3885516d,NaN,2719,2558,nofinding
235,14e9fceaa30d11ebb803f48e3885516d,NaN,2735,2802,nofinding
236,14e9fceba30d11eba2f4f48e3885516d,NaN,2368,2568,nofinding
237,14e9fceca30d11eb96cdf48e3885516d,NaN,2991,2950,nofinding
...,...,...,...,...,...
1317,14ed7f28a30d11eb905bf48e3885516d,NaN,2495,2736,nofinding
1318,14ed7f29a30d11ebb900f48e3885516d,NaN,2735,2802,nofinding
1319,14ed7f2aa30d11ebb8a8f48e3885516d,NaN,2801,2802,nofinding
1320,14ed7f2ba30d11eba5b1f48e3885516d,NaN,2619,2458,nofinding


In [ ]:
len(np.unique(df.file_name))

1322

In [ ]:
list_of_labels=df.class_name.to_numpy()
mask_path = df.file_name.to_numpy()

In [8]:
class_numbers = {'Background':0,
 'NoduloMasa': 1,
 'Consolidacion': 2,
 'PatronIntersticial': 3,
 'Atelectasia': 4,
 'LesionesDeLaPared': 5
 }

def getMaskBoxes(mask_path,list_of_labels):
    if isinstance(mask_path,str):
        # each color corresponds to a different instance with 0 being background
        with open(mask_path, 'rb') as f:
            mask = np.load(f)
        boxes = []
        labels = []
        raw_labels = [class_numbers[c] for c in list_of_labels.split('-')]
        # instances are encoded as different colors
        obj_ids = np.unique(mask)[1:]
        # split the color-encoded mask into a set of binary masks
        masks = mask == obj_ids[:, None, None]
        # get bounding box coordinates for each mask
        num_objs = len(obj_ids)
        for i in range(num_objs):
            pos = np.where(masks[i])
            xmin = np.min(pos[1])
            xmax = np.max(pos[1])
            ymin = np.min(pos[0])
            ymax = np.max(pos[0])
            if ymax > ymin and xmax > xmin:
                boxes.append([xmin, ymin, xmax, ymax])
                labels.append(raw_labels[i])            
        df = pd.DataFrame({'boxes':boxes,'labels':labels,'image':mask_path})
        return df
    else:
        return None

In [ ]:
results = [] 
for index,row in df.iterrows():
  results.append(getMaskBoxes('/content/drive/MyDrive/Quinto_Anio/TESIS_Eugenia_Berrino/Part_II_DS/DS_VAL_OD/unzipped/Euge/masks/'+row['file_name']+'.npy',row['class_name']))
  print(index)

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180


IndexError: ignored

In [ ]:
ans

,boxes,labels,image
0,"[1687, 1301, 1915, 1530]",1,/content/drive/MyDrive/Quinto_Anio/TESIS_Eugen...
1,"[366, 470, 1245, 1726]",2,/content/drive/MyDrive/Quinto_Anio/TESIS_Eugen...
